### 📖 Where We Are

**In the previous notebooks**, we explored two types of self-hosted vector stores:
1.  **ChromaDB**: A developer-friendly, local database perfect for prototyping.
2.  **FAISS**: A high-performance library for fast, in-memory similarity search.

**In this notebook**, we'll graduate to a **fully managed, cloud-native vector database: Pinecone**. This represents the type of infrastructure used in production-scale applications. We'll learn how to set up a Pinecone index, integrate it with LangChain, and see how the core RAG concepts remain the same even when using a powerful cloud service.

### 1. Building a RAG System with Langchain and Pinecone

### What is Pinecone?

**Pinecone** is a fully managed vector database designed for production environments. It provides a simple API to build high-performance, scalable AI applications that use vector search.

**Analogy: A Cloud-Based Digital Library Service ☁️**

- If **ChromaDB** is a local library and **FAISS** is a high-speed search index for that library, then **Pinecone** is a massive, cloud-based digital library service like Amazon's Kindle Store or Google Books.
- You don't manage the servers, storage, or infrastructure. You simply upload your documents (vectors) via an API, and the service handles the complexity of scaling, security, and providing ultra-fast search results to millions of users.

**Key Advantages:**
- **Fully Managed**: No need to worry about servers or infrastructure.
- **Highly Scalable**: Designed to handle billions of vectors with low latency.
- **Production-Ready**: Offers features like real-time indexing, metadata filtering, and high availability.

In [ ]:
# This cell is for securely storing your Pinecone API key.
# Replace "your pinecone api key" with your actual key from app.pinecone.io
api_key = "your_pinecone_api_key"

In [ ]:
# Install the necessary libraries for LangChain and Pinecone.
!pip install -qU langchain langchain-pinecone langchain-huggingface

In [ ]:
# Import the Pinecone client library and initialize it with your API key.
# This `pc` object is your main entry point for interacting with the Pinecone service.
from pinecone import Pinecone
pc = Pinecone(api_key=api_key)

In [ ]:
# We'll use a standard open-source model from Hugging Face for our embeddings.
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

### 2. Creating a Pinecone Index

In Pinecone, an **index** is a dedicated, isolated environment where your vectors are stored and searched. You need to create an index before you can add any data. The most critical step is ensuring the `dimension` of the index matches the output dimension of your embedding model.

In [ ]:
from pinecone import ServerlessSpec

# Define the name for your Pinecone index.
index_name = "rag-demo-index"

# Check if an index with this name already exists. This prevents errors on re-running the code.
if index_name not in pc.list_indexes().names():
    print(f"Creating index: {index_name}")
    # Create a new index.
    pc.create_index(
        name=index_name,
        # The dimension MUST match the embedding model's output dimension (384 for all-MiniLM-L6-v2).
        dimension=384,
        # We'll use cosine similarity, which is great for semantic search.
        metric="cosine",
        # `ServerlessSpec` defines the cloud infrastructure; it's a cost-effective, auto-scaling option.
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
else:
    print(f"Index '{index_name}' already exists.")

# Connect to the specific index we'll be working with.
index = pc.Index(index_name)

### 3. Integrating with LangChain
The `PineconeVectorStore` class is the LangChain wrapper that connects our Pinecone index to the standard `VectorStore` interface. This allows us to use it seamlessly in our RAG chains, just like we did with Chroma and FAISS.

In [ ]:
# Import the LangChain wrapper for Pinecone.
from langchain_pinecone import PineconeVectorStore

# Create a LangChain VectorStore object that is connected to our Pinecone index.
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
# Let's create some sample LangChain Document objects.
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Building an exciting new project with LangChain - come check it out!",
        metadata={"source": "tweet"},
    ),
    Document(
        page_content="Robbers broke into the city bank and stole $1 million in cash.",
        metadata={"source": "news"},
    ),
    Document(
        page_content="LangGraph is the best framework for building stateful, agentic applications!",
        metadata={"source": "tweet"},
    ),
    Document(
        page_content="The stock market is down 500 points today due to fears of a recession.",
        metadata={"source": "news"},
    )
]

In [ ]:
# The `.add_documents()` method handles the embedding and upserting process for us.
# 'Upserting' means it will update a vector if an ID already exists, or insert it if it's new.
vector_store.add_documents(documents=documents)
print("Documents have been added to the Pinecone index.")

### 4. Querying the Vector Store
Now that our vectors are stored in Pinecone, we can perform similarity searches. Managed databases like Pinecone have powerful features, including advanced metadata filtering, which allows you to narrow your search to only the documents that match certain criteria.

In [ ]:
# Perform a similarity search with a metadata filter.
results = vector_store.similarity_search(
    "What is LangChain?",  # The user's query.
    k=2,                     # The number of results to return.
    filter={"source": "tweet"} # Only search within documents where the source is 'tweet'.
)

print("Search results with metadata filter:")
for res in results:
    print(f"- {res.page_content} (Metadata: {res.metadata})")

### 📊 Vector Store Comparison

We've now seen three different types of vector stores, each with its own strengths.

| Vector Store | Type | Persistence | Scalability | Best For |
| :--- | :--- | :--- | :--- | :--- |
| **ChromaDB** | Local Database | Automatic (on disk) | Local Machine | Quick prototyping, local development, small to medium projects. |
| **FAISS** | In-Memory Library | Manual (`save`/`load`) | Single Machine RAM | High-speed, low-latency search on a single machine where the index fits in memory. |
| **Pinecone** | **Managed Cloud Service** | **Managed in Cloud** | **Cloud Scalable** | **Production applications, large-scale deployments, and when you don't want to manage infrastructure.** |

### 🔑 Key Takeaways

* **Pinecone is a Managed Service**: Pinecone is a fully managed, cloud-native vector database designed for production-level scalability and performance. You interact with it via an API.
* **Index is Key**: In Pinecone, you first create an **index**, which is a dedicated environment for your vectors. The `dimension` of this index **must** exactly match the output dimension of your embedding model.
* **LangChain Integration is Seamless**: The `PineconeVectorStore` wrapper allows you to use your Pinecone index as a standard LangChain `VectorStore`, making it easy to integrate into RAG pipelines with methods like `.add_documents` and `.similarity_search`.
* **Cloud-Native Benefits**: Managed databases like Pinecone excel at handling massive scale and provide powerful features like advanced metadata filtering without requiring you to manage any of the underlying infrastructure.